<a href="https://colab.research.google.com/github/lukaszlewickii/football_games_predictor/blob/main/eda_colab_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from skfeature.function.similarity_based import fisher_score
import torch
import xgboost as xgb
from sklearn.preprocessing import FunctionTransformer, StandardScaler

In [ ]:
!python --version

Python 3.10.12


In [ ]:
!nvidia-smi

Sun Aug  4 10:55:28 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
modelling_data = pd.read_csv('modelling_data.csv')

In [4]:
modelling_data.shape

(36085, 115)

In [5]:
modelling_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36085 entries, 0 to 36084
Columns: 115 entries, home_team_cumulative_goals_scored_pre_game to away_team_goal_count
dtypes: float64(101), int64(10), object(4)
memory usage: 31.7+ MB


In [6]:
modelling_data

,home_team_cumulative_goals_scored_pre_game,away_team_cumulative_goals_scored_pre_game,home_team_cumulative_goals_conceded_pre_game,away_team_cumulative_goals_conceded_pre_game,home_team_average_goals_scored_per_game_pre_game,away_team_average_goals_scored_per_game_pre_game,home_team_average_goals_conceded_per_game_pre_game,away_team_average_goals_conceded_per_game_pre_game,home_team_average_first_half_goals_scored_pre_game,home_team_average_second_half_goals_scored_pre_game,...,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,league,season,result,home_team_goal_count,away_team_goal_count
0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.69,2.75,4.75,1.83,2.00,austria,18/19,1,2,1
1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.71,2.80,4.25,1.95,1.87,austria,18/19,1,3,2
2,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.00,3.55,5.75,1.87,1.95,austria,18/19,2,2,3
3,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.80,3.05,4.25,2.10,1.77,austria,18/19,1,3,1
4,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.53,2.35,4.25,1.77,2.05,austria,18/19,2,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36080,29,21,36,31,1.000000,0.724138,1.241379,1.068966,0.551724,0.448276,...,2.38,4.10,7.50,1.95,1.75,ukraine,23/24,2,0,1
36081,37,60,30,22,1.275862,2.068966,1.034483,0.758621,0.448276,0.827586,...,1.80,2.98,5.60,1.89,1.81,ukraine,23/24,1,2,0
36082,18,27,38,35,0.620690,0.931034,1.310345,1.206897,0.275862,0.344828,...,2.37,4.40,8.80,2.20,1.63,ukraine,23/24,2,0,3
36083,48,27,30,47,1.655172,0.931034,1.034483,1.620690,0.827586,0.827586,...,1.74,3.18,6.00,1.78,1.93,ukraine,23/24,1,3,0


# Feature selection - wrapper methods

## Data preparation

In [21]:
X = modelling_data.drop(['league', 'season', 'result', 'home_team_goal_count', 'away_team_goal_count', 'round', 'time_of_day'], axis=1)
y = modelling_data['result']

In [13]:
def log_transform(X):
    return np.log(X + 1)

In [18]:
pd.set_option('display.max_rows', 150)

In [28]:
X_scaled = StandardScaler().fit_transform(X)

## Model

In [29]:
from tqdm import tqdm
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

In [37]:
torch.cuda.is_available()

True

In [40]:
!nvidia-smi

Sat Aug 10 11:44:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0              30W /  70W |    157MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [45]:
model = XGBClassifier(**{
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'objective': 'multi:softmax',
    'num_class': 3,
    'device': 'cuda'
})


In [41]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [42]:
rfe = RFE(estimator=model, n_features_to_select=10, step=1, verbose=0)

In [59]:
import warnings
warnings.filterwarnings('ignore')

In [58]:
rfe_results = {}

n_features = X_train.shape[1]
for i in tqdm(range(n_features, 0, -1)):
    rfe = RFE(estimator=model, n_features_to_select=i, step=1)
    rfe.fit(X_train, y_train)

    features_used = list(X.columns[rfe.support_])

    y_pred = rfe.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    tqdm.write(f" Number of features = {i}, Accuracy = {accuracy:.4f}")
    rfe_results[i] = {'features': features_used, 'accuracy': accuracy}

  1%|          | 1/108 [00:01<03:09,  1.77s/it]

 Number of features = 108, Accuracy = 0.5046


  2%|▏         | 2/108 [00:04<04:38,  2.63s/it]

 Number of features = 107, Accuracy = 0.5046


  3%|▎         | 3/108 [00:09<06:19,  3.61s/it]

...

 Number of features = 3, Accuracy = 0.5125


 99%|█████████▉| 107/108 [2:08:16<02:00, 120.19s/it]

 Number of features = 2, Accuracy = 0.5130


100%|██████████| 108/108 [2:10:18<00:00, 72.39s/it] 

 Number of features = 1, Accuracy = 0.5238


In [62]:
import json
with open('rfe_results.json', 'w') as f:
    json.dump(rfe_results, f)